In [1]:
import sys
import os
import geopandas as gpd
from osgeo import gdal
from shutil import copyfile, move

sys.path.insert(0, './HLB')
import hlb_utils

### Dataset

In [2]:
root_path = 'D:\FelipeSa\OneDrive\Felipe_INPE\Segmentacao\Analise_final\Sobradinho_areasul'

#canopy_dir = os.path.join(root_path, 'train_canopy_final.geojson')

rgb_dir = os.path.join(root_path, 'clippedrgb_12_05_2019_area_longe_transparent_mosaic_group1_eq.tif')
dem_dir = os.path.join(root_path, 'clippedrgb_12_05_2019_area_longe_chm.tif')

In [3]:
pre_processing = hlb_utils.PreProcessing(rgb_dir, dem_dir, proj_name='inferencia_sobradinho_areasul')

In [ ]:
pre_processing.orto = gdal.Open(os.path.join(root_path, 'RGB-D_0-255_stack.tif'))  # Dataset da imagem
#pre_processing.pxsize = round(pre_processing.orto.GetGeoTransform()[1], 5)  # Tamanho do pixel (m)
round(pre_processing.orto.GetGeoTransform()[1], 5)

### Merge RGB and DEM

In [4]:
pre_processing.merge_rgb_dem()

Reading source image information
Writing output file
Band 1
Band 2
Band 3
Band 4
Setting options to output file
Done!


### Split image [and vectors]

In [7]:
pre_processing.split_img(patch_size=512, overlap=100, save_img=True)

Cortando imagens: 100%|████████████████████████████████████████████████████████████| 1240/1240 [01:10<00:00, 17.58it/s]


Finished!
Total images:  858


In [ ]:
pre_processing.split_vector()

### Make annotation

In [4]:
'../datasets/segmentacao_aguai'
'../datasets/segmentacao_carmona'
'../datasets/segmentacao_sobradinho_sede'

pre_processing.proj_dir = '../datasets/segmentacao_todos'
# pre_processing.patch_size = 512
# pre_processing.pxsize = 0.025

In [5]:
pre_processing.grid = gpd.read_file('../datasets/segmentacao_todos/img_grid.geojson').set_index('id')

In [6]:
dict_ref = {range(1,727): [512, 0.025], range(727,1392): [512, 0.02216], range(1392, 1716): [256, 0.07106]}

pre_processing.make_annotations_joined(dict_ref)

### Outros

In [ ]:
import os
from osgeo import gdal, osr
from PIL import Image

shape_path_aux = '../IMGS/TALHAO_ANGULADAS/INF'

result_paths = []
for filename in os.listdir(shape_path_aux):
    result_paths.append(os.path.join(shape_path_aux, filename))

for path in result_paths:
    
    img = Image.open(path)
    w, h = img.size
    img = img.crop((1000, 700, w, h))
    out_path = os.path.join(os.path.dirname(os.path.dirname(path)), 'EDITS', os.path.basename(path))
    img.save(out_path)

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from time import sleep


shape_path_aux = '../datasets/ortofoto_talhao/results'

with tqdm(total=len(os.listdir(shape_path_aux)) as pbar:
    pbar.set_description("Lendo arquivos de entradas")
    sleep(0.1)

    result_paths = []
    for filename in os.listdir(shape_path_aux):
        if os.path.splitext(filename)[1].lower() == '.geojson':
            result_paths.append(os.path.join(shape_path_aux, filename))
        pbar.update(1)
    

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols['geometry'] = pols.buffer(-0.1).buffer(0.1).simplify(0.03)
pols = pols.explode().reset_index(drop=True)
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= 1]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index
# pols['centroid'] = pols['geometry'].centroid


over = gpd.overlay(pols, pols, how='intersection')
over = over.loc[-over['geometry'].is_empty]
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)
delete_lines = []
with tqdm(total=len(over['id_1'])) as pbar:
    pbar.set_description("Simplificando - Micro")
    sleep(0.1)

    for i in over['id_1']:

        intersec = over[over['id_1']==i]

        if intersec.loc[intersec['geometry'].area >= 0.6 * pols.loc[i, 'geometry'].area].any(axis=None) or \
        (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
            delete_lines.append(i)
            
        pbar.update(1)

    delete_lines = (list(set(delete_lines)))

pols.drop(labels=delete_lines, axis=0, inplace=True)
pols.explode().reset_index(drop=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols.to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result.geojson'), driver='GeoJSON')

In [ ]:
pols['id'] = pols.index
pols

In [ ]:
pols.to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result_1.geojson'), driver='GeoJSON')

In [ ]:
over = gpd.overlay(pols, pols, how='intersection')
over = over.loc[-over['geometry'].is_empty]
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)

In [ ]:
over

In [ ]:
delete_lines = []

for i in over['id_1']:
    
    intersec = over[over['id_1']==i]
    
    if intersec.loc[intersec['geometry'].area >= 0.6 * pols.loc[i, 'geometry'].area].any(axis=None) or \
    (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
        delete_lines.append(i)
        
delete_lines = (list(set(delete_lines)))
delete_lines.sort()            

In [ ]:
pols.loc[delete_lines].to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_delete.geojson'), driver='GeoJSON')

In [ ]:
pols.drop(labels=delete_lines, axis=0, inplace=True)
pols

In [ ]:
os.path.dirname(shape_path_aux)

In [ ]:
pols.buffer(-0.1).buffer(0.1).simplify(0.03).to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result_faster_simplify.geojson'), driver='GeoJSON')

In [ ]:
import os
import rasterio
import numpy as np
from osgeo import gdal

rgb_path = '/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/rgb_20_12_19_Voo1e2_transparent_mosaic_group1_5cm_modified.tif'
dem_path = '/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/rgb_20_12_19_Voo1e2_dsm_5cm_modified.tif'

# Read metadata of first file
with rasterio.open(rgb_path) as src_rgb:
    meta_dst = src_rgb.meta

# # Update meta to reflect the number of layers
meta_dst.update(count = 4)

# # Read each layer and write it to stack
with rasterio.open('/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/stack.tif', 'w', **meta_dst) as dst:

    for band in range(1,4):
        with rasterio.open(rgb_path) as src_rgb:
            dst.write_band(band, src_rgb.read(band))

    with rasterio.open(dem_path) as src_dem:

        meta_dem = src_dem.meta

        band_dem = src_dem.read(1)
        band_dem[band_dem == meta_dem['nodata']] = np.nan
        min_value, max_value = np.nanmin(band_dem), np.nanmax(band_dem)

        band_dem_rescaled = np.interp(band_dem, (min_value, max_value), (0, 255))

        band_dem_rescaled[band_dem_rescaled == np.nan] = 0

        dst.write_band(4, band_dem_rescaled.astype('uint8'))

ds = gdal.Open('/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/stack.tif')
gdal.Translate('/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/stack_final.tif', ds, creationOptions = ['ALPHA=NO'])
os.remove('/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/stack.tif')
ds = None


In [ ]:
band_dem

In [ ]:
band_dem.astype('uint8')

In [ ]:
import rasterio
import numpy as np

rgb_path = '/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/rgb_20_12_19_Voo1e2_transparent_mosaic_group1_5cm_modified.tif'
dem_path = '/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/rgb_20_12_19_Voo1e2_dsm_5cm_modified.tif'


with rasterio.open(dem_path) as src_dem:
    
    meta = src_dem.meta
    
    band_dem = src_dem.read(1)
    band_dem[band_dem == meta['nodata']] = np.nan
    min_value, max_value = np.nanmin(band_dem), np.nanmax(band_dem)
    
    band_dem_rescaled = np.interp(band_dem, (min_value, max_value), (0, 255))
    
    band_dem_rescaled[band_dem_rescaled == meta['nodata']] = np.nan
    band_dem_rescaled.astype
    
# band_dem

In [ ]:
band_dem

In [ ]:
np.nanmax(np.interp(band_dem, (min_value, max_value), (0, 255)))

In [ ]:
meta

In [ ]:
import numpy as np
band_dem[band_dem == meta['nodata']] = np.nan

In [ ]:
band_dem.mean()

In [ ]:
np.nanmin(band_dem)

In [ ]:
os.path.dirname('/mnt/7F93AA61350F3FC5/Images_VANT/Campo_20_12_19/RGB/modified/rgb_20_12_19_Voo1e2_transparent_mosaic_group1_5cm_modified.tif')